In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Part 1

In [70]:
import numpy as np
from copy import deepcopy

class PathState:
    def __init__(self,loc,steps):
        self.loc = loc
        self.steps = steps

    def give_neighbors(self,garden):
        neighs=[]
        nR,nC = len(garden),len(garden[0])
        
        changes = [ (-1,0), (1,0), (0,1), (0,-1) ]
        for xc in changes:
            new_loc = tuple([ i+j for i,j in zip(xc,self.loc)])
        
            if (new_loc[0] > -1 and new_loc[1] > -1  
                and new_loc[0] < nR and new_loc[1] < nC):
                
                if garden[new_loc[0]][new_loc[1]] != '#':
                    neighs.append(PathState(new_loc, self.steps+1))
        
        return neighs
                   
    def __repr__(self):
        return f'{self.loc[0]}-{self.loc[1]}:{self.steps}'
    
    def __str__(self):
        return f'{self.loc[0]}-{self.loc[1]}:{self.steps}'
    
    

In [71]:
from collections import deque
total=64
def perform_breadth_first(garden):
    frontier = deque()
    start = None
    for ir, r in enumerate(garden):
        if "S" in r:
            ic = r.find('S')
            start = (ir, ic) 
            break
    
    print(start)
    frontier.append(PathState(start,0))
    
    
    reached_req =[]
    seen =[]
    
    while len(frontier) > 0  :
        cur_loc = frontier.popleft()
        if cur_loc.steps ==total:
            reached_req.append(cur_loc.loc)
        
        else:
            for n_loc in cur_loc.give_neighbors(garden):
                if str(n_loc) not in seen:
                    seen.append(str(n_loc))
                    frontier.append(n_loc)
                    
    return len(set(reached_req))

In [72]:
def do_part_one(file_path):
    with open(file_path,'r') as f:
        garden = [l.strip() for l in f.readlines()]
    
    nlocs = perform_breadth_first(garden)
    return nlocs

In [73]:
%%time
tmp = do_part_one('input_data/test_21.txt')
tmp

(5, 5)
CPU times: user 108 ms, sys: 2.41 ms, total: 111 ms
Wall time: 113 ms


42

In [74]:
%%time
do_part_one('input_data/day_21.txt')

(65, 65)
CPU times: user 1min 22s, sys: 29 ms, total: 1min 22s
Wall time: 1min 23s


3651

# Part 2

In [85]:
def do_part_two(file_path):
    with open(file_path,'r') as f:
        heat_loss = [ [int(i) for i in l.strip()] for l in f.readlines()]
    
    heat = ultimate_path(heat_loss)
    nR,nC = len(heat_loss), len(heat_loss[0])
                                          
    blank = [ ['.']*nC for r in range(nR)]

    spot = heat
    while spot is not None:
        loc = spot.loc
        blank[loc[0]][loc[1]] = spot.direction
        spot = spot.came_from
    
    
    print('\n'.join([ ''.join(row) for row in blank]))
 #   
    return heat

In [ ]:
%%time
do_part_two('input_data/test_17.txt')
           

In [ ]:
%%time
do_part_two('input_data/day_17.txt')
           